# 📄 Free Local CV & Bid Match Analyzer (No API, 100% Private)
Analyze how well your CV matches a bid or job advert using local models.

✅ No API keys required
✅ Supports PDF & DOCX
✅ Outputs match score, missing keywords, and suggestions

### 🔧 How to Use:
1. Click `Runtime > Run all`
2. Upload your CV
3. Upload the bid/job advert
4. View analysis and download report

In [ ]:
# Install required libraries
!pip install -q sentence-transformers transformers docx2txt pdfplumber fpdf

In [ ]:
# Imports
import pdfplumber, docx2txt
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch
import os
from fpdf import FPDF
from google.colab import files


In [ ]:
# Upload CV and Bid Advert
from google.colab import files
uploaded = files.upload()
cv_path = list(uploaded.keys())[0]

uploaded = files.upload()
bid_path = list(uploaded.keys())[0]

In [ ]:
# Extract text
def extract_text(path):
    if path.endswith('.pdf'):
        with pdfplumber.open(path) as pdf:
            return ' '.join(page.extract_text() or '' for page in pdf.pages)
    elif path.endswith('.docx'):
        return docx2txt.process(path)
    return ''

cv_text = extract_text(cv_path)
bid_text = extract_text(bid_path)

In [ ]:
# Semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')
cv_emb = model.encode(cv_text, convert_to_tensor=True)
bid_emb = model.encode(bid_text, convert_to_tensor=True)
score = util.cos_sim(cv_emb, bid_emb).item() * 100
print(f"Match Score: {score:.2f}%")

In [ ]:
# Keyword extraction
extractor = pipeline('text2text-generation', model='t5-small')
bid_keywords = extractor(f"extract keywords: {bid_text[:512]}")[0]['generated_text']
cv_keywords = extractor(f"extract keywords: {cv_text[:512]}")[0]['generated_text']

print("\nBid Keywords:", bid_keywords)
print("\nYour CV Keywords:", cv_keywords)

In [ ]:
# Simple keyword gap analysis
bid_keys = set(bid_keywords.lower().split(', '))
cv_keys = set(cv_keywords.lower().split(', '))
missing = bid_keys - cv_keys

print("\nMissing Keywords:", missing)

In [ ]:
# Generate PDF report
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 12)
pdf.cell(0, 10, 'CV & Bid Match Report', ln=True)
pdf.set_font('Arial', '', 11)
pdf.multi_cell(0, 10, f"Match Score: {score:.2f}%\n\nMissing Keywords: {', '.join(missing)}")
pdf_path = "cv_bid_report.pdf"
pdf.output(pdf_path)
files.download(pdf_path)